In [1]:
from stable_baselines3 import PPO
from minedojo.sim import InventoryItem
import gym

import numpy as np
import minedojo


[INFO:minedojo.tasks] Loaded 1581 Programmatic tasks, 1560 Creative tasks, and 1 special task: "Playthrough". Totally 3142 tasks loaded.


In [2]:
class MineDojoRGBWrapper(gym.ObservationWrapper):
    def __init__(self, env):
        super().__init__(env)

        # assuming image_size=84
        self.observation_space = gym.spaces.Box(
            low=0, high=255, shape=(3, 84, 84), dtype=np.uint8
        )

    def observation(self, obs):
        rgb = obs["rgb"]  # strip EVERYTHING else
        rgb = np.array(rgb)
        return rgb.astype(np.uint8)
    def step(self, action):
        try:
            return self.env.step(action)
        except ValueError:
            # Invalid action → treat as no-op
            obs = self.env.observation_space.sample()
            reward = -1  # small penalty
            done = False
            info = {}
            return obs, reward, done, info

In [3]:
env = minedojo.make(
    "open-ended",
    image_size=84,   
    initial_inventory=[
        InventoryItem(slot=0, name="flint_and_steel", quantity=1),
        InventoryItem(slot=1, name="obsidian", quantity=20),
    ],
    spawn_in_village=True
)

env = MineDojoRGBWrapper(env)


/home/john/miniconda3/envs/cs175/lib/python3.11/site-packages/numpy/core/numeric.py:330: RuntimeWarning: invalid value encountered in cast
  multiarray.copyto(a, fill_value, casting='unsafe')


In [ ]:
obs = env.reset()
print(type(obs), obs.shape)

[INFO:minedojo.tasks] Loaded 1581 Programmatic tasks, 1560 Creative tasks, and 1 special task: "Playthrough". Totally 3142 tasks loaded.


<class 'numpy.ndarray'> (3, 84, 84)


: 

In [ ]:
model = PPO(policy='MlpPolicy', env=env, verbose=1)
model.learn(total_timesteps=1_000_000)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [ ]:
env.reset()

In [ ]:
env.close()